In [2]:
import json

with open('/app/Law_corpus/documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [3]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("truro7/vn-law-embedding", truncate_dim = 768) #truncate_dim = 768 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.07k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

Invalid model-index. Not loading eval results into CardData.


1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Invalid model-index. Not loading eval results into CardData.


In [4]:
v = model.encode('Người lái xe cần tuân thủ quy định về tốc độ như thế nào trên đường?')

In [5]:
len(v)

768

In [7]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

# es_client.info()
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "law_title": {"type": "keyword"},                
            "law_number": {"type": "keyword"},               
            "chapter_title": {"type": "keyword"},            
            "article_number": {"type": "keyword"},            
            "title": {"type": "text"},                        
            "content": {"type": "text"},   
            "id": {"type": "text"},   
            "title_vector": {                                 
                "type": "dense_vector",
                "dims": 768,                                 
                "index": True,
                "similarity": "cosine"
            },
            "content_vector": {                                 
                "type": "dense_vector",
                "dims": 768,                                 
                "index": True,
                "similarity": "cosine"
            },
            "title_content_vector": {                                 
                "type": "dense_vector",
                "dims": 768,                                 
                "index": True,
                "similarity": "cosine"
            }                
        }
    }
}

index_name = "legal_documents_ids"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7fad655d3d30>: Failed to establish a new connection: [Errno 111] Connection refused))

In [46]:
from tqdm.auto import tqdm

In [47]:
for doc in tqdm(documents):
    title = doc['title']
    content = doc['content']
    qt = title + ' ' + content

    doc['title_vector'] = model.encode(title)
    doc['content_vector'] = model.encode(content)
    doc['title_content_vector'] = model.encode(qt)

  0%|          | 0/314 [00:00<?, ?it/s]

In [48]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/314 [00:00<?, ?it/s]

In [51]:
query = 'Khi nào thì người điều khiển được phép lùi xe?'

In [52]:
v_q = model.encode(query)

In [56]:
documents[-1]

{'law_title': 'Luật Giao Thông Đường Bộ',
 'law_number': 'Luật số: 23/2008/QH12',
 'chapter_title': 'CHƯƠNG VIII',
 'article_number': 'Điều 89',
 'title': 'Quy định chi tiết và hướng dẫn thi hành',
 'content': 'Chính phủ và cơ quan có thẩm quyền quy định chi tiết và hướng dẫn thi hành các điều, khoản được giao trong Luật; hướng dẫn những nội dung cần thiết khác của Luật này để đáp ứng yêu cầu quản lý nhà nước. Luật này đã được Quốc hội nước Cộng hòa xã hội chủ nghĩa Việt Nam khóa XII, kỳ họp thứ 4 thông qua ngày 13 tháng 11 năm 2008. CHỦ TỊCH QUỐC HỘI Nguyễn Phú Trọng ',
 'id': '2bb7dafa',
 'title_vector': array([-7.99200905e-04,  5.17771719e-03,  1.00677451e-02,  4.25530188e-02,
        -7.46648088e-02, -1.46973412e-02,  7.89543847e-04, -3.56543213e-02,
        -1.54640134e-02, -1.49523607e-02, -3.37247029e-02, -9.68723965e-04,
        -4.36103866e-02,  1.64200701e-02, -8.93867016e-03,  1.31670712e-02,
        -5.95484897e-02,  7.83475041e-02, -5.42611291e-04, -6.18260028e-03,
       

In [55]:
def elastic_search_knn(field, vector, law_title):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "law_title": law_title
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["law_title", "law_number", "chapter_title", "article_number", "title", "content","id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [57]:
elastic_search_knn('content_vector',v_q,'Luật Giao Thông Đường Bộ')

[{'article_number': 'Điều 16',
  'law_number': 'Luật số: 23/2008/QH12',
  'chapter_title': 'CHƯƠNG II',
  'id': '7f7a2ff6',
  'title': 'Lùi xe',
  'law_title': 'Luật Giao Thông Đường Bộ',
  'content': '1. Khi lùi xe, người điều khiển phải quan sát phía sau, có tín hiệu cần thiết và chỉ khi nào thấy không nguy hiểm mới được lùi. 2. Không được lùi xe ở khu vực cấm dừng, trên phần đường dành cho người đi bộ qua đường, nơi đường bộ giao nhau, đường bộ giao nhau cùng mức với đường sắt, nơi tầm nhìn bị che khuất, trong hầm đường bộ, đường cao tốc. '},
 {'article_number': 'Điều 27',
  'law_number': 'Luật số: 23/2008/QH12',
  'chapter_title': 'CHƯƠNG II',
  'id': '4f15f43a',
  'title': 'Giao thông trong hầm đường bộ',
  'law_title': 'Luật Giao Thông Đường Bộ',
  'content': 'Người điều khiển phương tiện trong hầm đường bộ ngoài việc tuân thủ các quy tắc giao thông quy định tại Luật này còn phải thực hiện các quy định sau đây: 1. Xe cơ giới, xe máy chuyên dùng phải bật đèn; xe thô sơ phải bật đè

In [59]:
def question_vector_knn(q):
    questions = q['questions']
    law_title = q['law_title']

    v_q = model.encode(questions)

    return elastic_search_knn('content_vector', v_q, law_title)

In [60]:
import pandas as pd

In [61]:
df_ground_truth = pd.read_csv('./ground-truth-data.csv')

In [62]:
ground_truth = df_ground_truth.to_dict(orient='records')

In [64]:
ground_truth[-1]

{'questions': 'Luật có liên quan đến những nội dung nào khác không?',
 'law_title': 'Luật Giao Thông Đường Bộ',
 'document': '2bb7dafa'}

In [65]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [66]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [67]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [68]:
evaluate(ground_truth, question_vector_knn)

  0%|          | 0/1570 [00:00<?, ?it/s]

{'hit_rate': 0.9063694267515924, 'mrr': 0.7849044585987267}

ES text only: 0.7395720769397017, 0.6032418413658963

In [77]:
def title_content_vector_knn(q):
    questions = q['questions']
    law_title = q['law_title']

    v_q = model.encode(questions)

    return elastic_search_knn('title_content_vector', v_q, law_title)


In [ ]:
evaluate(ground_truth, title_content_vector_knn)

  0%|          | 0/1570 [00:00<?, ?it/s]

{'hit_rate': 0.9159235668789809, 'mrr': 0.7966985138004254}

: 